# Proyecto ATP-Analisis-Ranking
## Descripción
Este proyecto tiene como objetivo analizar diversas estadísticas de la base de datos de la ATP (Asociación de Tenistas Profesionales) y construir un modelo personalizado de
ranking basado en el rendimiento de los jugadores. Posteriormente, compararemos nuestro ranking con el ranking actual de la ATP para determinar la efectividad de nuestro modelo.

In [831]:
# IMPORT LIBRARIES

import pandas as pd
import numpy as np
import plotly.express as px


In [832]:
# READ DATA

df_serve = pd.read_csv('Serve _ATP_Tennis.csv')
df_return = pd.read_csv('return_ ATP_Tennis.csv')
df_underpressure = pd.read_csv('Underpressure_ ATP_Tennis.csv')
df_win_loss = pd.read_csv('Win_Loss_ ATP_Tennis.csv')
df_ranking = pd.read_csv('ATP_Ranking.csv')

In [833]:
## DATA PREPROCESSING - df_serve

### Nos quedamos solo con las columnas que nos interesan.

df_serve = df_serve[["ServeRating©2", "_1stServe", "_1stServePointsWon", "_2ndServePointsWon", "_ServiceGamesWon", "Avg\\.Aces_Match", "Avg\\.DoubleFaults_Match"]]
df_serve.head()

,ServeRating©2,_1stServe,_1stServePointsWon,_2ndServePointsWon,_ServiceGamesWon,Avg\.Aces_Match,Avg\.DoubleFaults_Match
0,Nick Kyrgios,308.6,67.4%,79.4%,57.6%,92.9%,14.8
1,John Isner,308.5,67.7%,79.8%,52.7%,90.2%,21.1
2,Hubert Hurkacz,299.7,63.4%,79.2%,54.9%,89.9%,14.0
3,Matteo Berrettini,292.2,62.5%,79.6%,52.0%,88.5%,11.5
4,Novak Djokovic,292.1,65.3%,77.2%,57.0%,88.7%,6.0


In [834]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_serve.rename(columns={"ServeRating©2": "Player", "_1stServe": "1stServe", "_1stServePointsWon": "1stServePointsWon", "_2ndServePointsWon": "2ndServePointsWon", "_ServiceGamesWon": "ServiceGamesWon", "Avg\\.Aces_Match": "AvgAcesMatch", "Avg\\.DoubleFaults_Match": "AvgDoubleFaultsMatch"}, inplace=True)
df_serve.head()

,Player,1stServe,1stServePointsWon,2ndServePointsWon,ServiceGamesWon,AvgAcesMatch,AvgDoubleFaultsMatch
0,Nick Kyrgios,308.6,67.4%,79.4%,57.6%,92.9%,14.8
1,John Isner,308.5,67.7%,79.8%,52.7%,90.2%,21.1
2,Hubert Hurkacz,299.7,63.4%,79.2%,54.9%,89.9%,14.0
3,Matteo Berrettini,292.2,62.5%,79.6%,52.0%,88.5%,11.5
4,Novak Djokovic,292.1,65.3%,77.2%,57.0%,88.7%,6.0


In [835]:
### Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_serve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Player                77 non-null     object 
 1   1stServe              77 non-null     float64
 2   1stServePointsWon     77 non-null     object 
 3   2ndServePointsWon     77 non-null     object 
 4   ServiceGamesWon       77 non-null     object 
 5   AvgAcesMatch          77 non-null     object 
 6   AvgDoubleFaultsMatch  77 non-null     float64
dtypes: float64(2), object(5)
memory usage: 4.3+ KB


Como podemos ver, tenemos 5 columnas con datos object y 3 columnas con datos Float64. Además, no tenemos valores nulos en ninguna columna.
Lo que vamos a hacer es cambiar todas las columnas con datos object a tipo float, excepto la primera columna que contiene el nombre del jugador.

In [836]:
# Creamos una funcion que nos sirva para todos los dataframes.

def convert_to_float(df):
    """
    Esta función elimina el símbolo de porcentaje y convierte todos los valores en un DataFrame a tipo 'float',
    dividiendo por 100, excepto la primera columna.

    :param df: DataFrame de entrada
    :return: DataFrame con los valores modificados
    """
    for column in df.columns[1:]:
        df[column] = df[column].apply(lambda x: str(x).replace("%", "") if isinstance(x, str) else x)
        df[column] = pd.to_numeric(df[column], errors='coerce', downcast='float')

    return df



In [837]:
# convertimos todas las columnas a float

df_serve = convert_to_float(df_serve)
df_serve.head()

,Player,1stServe,1stServePointsWon,2ndServePointsWon,ServiceGamesWon,AvgAcesMatch,AvgDoubleFaultsMatch
0,Nick Kyrgios,308.600006,67.400002,79.400002,57.599998,92.900002,14.8
1,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1
2,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0
3,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5
4,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0


In [838]:
# Comprobamos que los datos se han convertido correctamente.

df_serve.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Player                77 non-null     object 
 1   1stServe              77 non-null     float32
 2   1stServePointsWon     77 non-null     float32
 3   2ndServePointsWon     77 non-null     float32
 4   ServiceGamesWon       77 non-null     float32
 5   AvgAcesMatch          77 non-null     float32
 6   AvgDoubleFaultsMatch  77 non-null     float32
dtypes: float32(6), object(1)
memory usage: 2.5+ KB


In [839]:
# DATA PREPROCESSING - df_return

### Nos quedamos solo con las columnas que nos interesan.

df_return = df_return[["ReturnRating©2", "_1stServeReturnPointsWon", "_2ndServeReturnPointsWon",  "_ReturnGamesWon", "_BreakPointsConverted"]]
df_return.head()

,ReturnRating©2,_1stServeReturnPointsWon,_2ndServeReturnPointsWon,_ReturnGamesWon,_BreakPointsConverted
0,Bernabe Zapata Miralles,165.3,34.2%,52.3%,31.0%
1,Daniil Medvedev,164.6,32.8%,55.2%,30.6%
2,Rafael Nadal,160.8,34.4%,51.7%,30.0%
3,Carlos Alcaraz,160.6,34.4%,53.5%,31.4%
4,Novak Djokovic,160.5,33.5%,55.9%,29.3%


In [840]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_return.rename(columns={"ReturnRating©2": "Player", "_1stServeReturnPointsWon": "1stServeReturnPointsWon", "_2ndServeReturnPointsWon": "2ndServeReturnPointsWon", "_ReturnGamesWon": "ReturnGamesWon", "_BreakPointsConverted": "BreakPointsConverted_return"}, inplace=True)
df_return.head()

,Player,1stServeReturnPointsWon,2ndServeReturnPointsWon,ReturnGamesWon,BreakPointsConverted_return
0,Bernabe Zapata Miralles,165.3,34.2%,52.3%,31.0%
1,Daniil Medvedev,164.6,32.8%,55.2%,30.6%
2,Rafael Nadal,160.8,34.4%,51.7%,30.0%
3,Carlos Alcaraz,160.6,34.4%,53.5%,31.4%
4,Novak Djokovic,160.5,33.5%,55.9%,29.3%


In [841]:
### Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_return.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Player                       77 non-null     object 
 1   1stServeReturnPointsWon      77 non-null     float64
 2   2ndServeReturnPointsWon      77 non-null     object 
 3   ReturnGamesWon               77 non-null     object 
 4   BreakPointsConverted_return  77 non-null     object 
dtypes: float64(1), object(4)
memory usage: 3.1+ KB


In [842]:
# convertimos todas las columnas a float

df_return = convert_to_float(df_return)
df_return.head()

,Player,1stServeReturnPointsWon,2ndServeReturnPointsWon,ReturnGamesWon,BreakPointsConverted_return
0,Bernabe Zapata Miralles,165.300003,34.200001,52.299999,31.000000
1,Daniil Medvedev,164.600006,32.799999,55.200001,30.600000
2,Rafael Nadal,160.800003,34.400002,51.700001,30.000000
3,Carlos Alcaraz,160.600006,34.400002,53.500000,31.400000
4,Novak Djokovic,160.500000,33.500000,55.900002,29.299999


In [843]:
# Comprobamos que los datos se han convertido correctamente.

df_return.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Player                       77 non-null     object 
 1   1stServeReturnPointsWon      77 non-null     float32
 2   2ndServeReturnPointsWon      77 non-null     float32
 3   ReturnGamesWon               77 non-null     float32
 4   BreakPointsConverted_return  77 non-null     float32
dtypes: float32(4), object(1)
memory usage: 1.9+ KB


In [844]:
# DATA PREPROCESSING - df_underpressure

### Nos quedamos solo con las columnas que nos interesan.

df_underpressure = df_underpressure[["UnderPressureRating©2", "_BreakPointsConverted", "_BreakPointsSaved", "_TieBreaksWon", "_DecidingSetsWon"]]
df_underpressure.head()

,UnderPressureRating©2,_BreakPointsConverted,_BreakPointsSaved,_TieBreaksWon,_DecidingSetsWon
0,Novak Djokovic,249.7,41.8%,64.3%,80.0%
1,Nick Kyrgios,249.7,43.4%,74.4%,61.9%
2,Jenson Brooksby,246.9,46.3%,58.9%,75.0%
3,Jannik Sinner,238.6,43.3%,63.5%,68.2%
4,Stefanos Tsitsipas,237.7,41.8%,66.0%,56.8%


In [845]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_underpressure.rename(columns={"UnderPressureRating©2": "Player", "_BreakPointsConverted": "BreakPointsConverted_pressure", "_BreakPointsSaved": "BreakPointsSavedPressure", "_TieBreaksWon": "TieBreaksWon", "_DecidingSetsWon": "DecidingSetsWon"}, inplace=True)
df_underpressure.head()

,Player,BreakPointsConverted_pressure,BreakPointsSavedPressure,TieBreaksWon,DecidingSetsWon
0,Novak Djokovic,249.7,41.8%,64.3%,80.0%
1,Nick Kyrgios,249.7,43.4%,74.4%,61.9%
2,Jenson Brooksby,246.9,46.3%,58.9%,75.0%
3,Jannik Sinner,238.6,43.3%,63.5%,68.2%
4,Stefanos Tsitsipas,237.7,41.8%,66.0%,56.8%


In [846]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_underpressure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Player                         77 non-null     object 
 1   BreakPointsConverted_pressure  77 non-null     float64
 2   BreakPointsSavedPressure       77 non-null     object 
 3   TieBreaksWon                   77 non-null     object 
 4   DecidingSetsWon                77 non-null     object 
dtypes: float64(1), object(4)
memory usage: 3.1+ KB


In [847]:
# convertimos todas las columnas a float

df_underpressure = convert_to_float(df_underpressure)
df_underpressure.head()

,Player,BreakPointsConverted_pressure,BreakPointsSavedPressure,TieBreaksWon,DecidingSetsWon
0,Novak Djokovic,249.699997,41.799999,64.300003,80.000000
1,Nick Kyrgios,249.699997,43.400002,74.400002,61.900002
2,Jenson Brooksby,246.899994,46.299999,58.900002,75.000000
3,Jannik Sinner,238.600006,43.299999,63.500000,68.199997
4,Stefanos Tsitsipas,237.699997,41.799999,66.000000,56.799999


In [848]:
# Comprobamos que los datos se han convertido correctamente.

df_underpressure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Player                         77 non-null     object 
 1   BreakPointsConverted_pressure  77 non-null     float32
 2   BreakPointsSavedPressure       77 non-null     float32
 3   TieBreaksWon                   77 non-null     float32
 4   DecidingSetsWon                77 non-null     float32
dtypes: float32(4), object(1)
memory usage: 1.9+ KB


In [849]:
df_win_loss.head()

,Player,YTDIndex_URL,YTDIndex,YTDTitles,YTDWin_Loss,Field,Field1,Field2
0,1,https://www.atptour.com/en/players/novak-djoko...,https://www.atptour.com/-/media/alias/player-h...,https://www.atptour.com/en/~/media/images/flag...,Novak Djokovic,0.938,\n\t\t\t\t\t\t\t\t\t2\n\t\t\t\t\t\t\t\t,15-1
1,2,https://www.atptour.com/en/players/carlos-alca...,https://www.atptour.com/-/media/alias/player-h...,https://www.atptour.com/en/~/media/images/flag...,Carlos Alcaraz,0.933,\n\t\t\t\t\t\t\t\t\t2\n\t\t\t\t\t\t\t\t,14-1
2,3,https://www.atptour.com/en/players/daniil-medv...,https://www.atptour.com/-/media/alias/player-h...,https://www.atptour.com/en/~/media/images/flag...,Daniil Medvedev,0.889,\n\t\t\t\t\t\t\t\t\t3\n\t\t\t\t\t\t\t\t,24-3
3,4,https://www.atptour.com/en/players/cameron-nor...,https://www.atptour.com/-/media/alias/player-h...,https://www.atptour.com/en/~/media/images/flag...,Cameron Norrie,0.840,\n\t\t\t\t\t\t\t\t\t1\n\t\t\t\t\t\t\t\t,21-4
4,5,https://www.atptour.com/en/players/nicolas-jar...,https://www.atptour.com/-/media/alias/player-h...,https://www.atptour.com/en/~/media/images/flag...,Nicolas Jarry,0.833,\n\t\t\t\t\t\t\t\t\t1\n\t\t\t\t\t\t\t\t,10-2


In [850]:
# DATA PREPROCESSING - df_win_loss

### Nos quedamos solo con las columnas que nos interesan.

df_win_loss = df_win_loss[["YTDWin_Loss", "Field"]]
df_win_loss.head()

,YTDWin_Loss,Field
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [851]:
### Hacemos un rename en el nombre de las columnas para que sean más intuitivas.

df_win_loss.rename(columns={"YTDWin_Loss": "Player", "Field": "AvgWinLoss"}, inplace=True)
df_win_loss.head()

,Player,AvgWinLoss
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [852]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      90 non-null     object 
 1   AvgWinLoss  90 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.5+ KB


In [853]:
# convertimos todas las columnas a float

df_win_loss = convert_to_float(df_win_loss)
df_win_loss.head()

,Player,AvgWinLoss
0,Novak Djokovic,0.938
1,Carlos Alcaraz,0.933
2,Daniil Medvedev,0.889
3,Cameron Norrie,0.840
4,Nicolas Jarry,0.833


In [854]:
# Comprobamos que los datos se han convertido correctamente.

df_win_loss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      90 non-null     object 
 1   AvgWinLoss  90 non-null     float32
dtypes: float32(1), object(1)
memory usage: 1.2+ KB


In [855]:
# DATA PREPROCESSING - df_ranking

### Nos quedamos solo con las columnas que nos interesan.

df_ranking = df_ranking[["Rank", "Player", "Age"]]

df_ranking.head()

,Rank,Player,Age
0,\n 1\n,\n Carlos Alcaraz\n...,\n19
1,\n 2\n,\n Novak Djokovic\n...,\n35
2,\n 3\n,\n Stefanos Tsitsip...,\n24
3,\n 4\n,\n Casper Ruud\n ...,\n24
4,\n 5\n,\n Daniil Medvedev\...,\n27


In [856]:
# Vamos a ver que tipo de datos tenemos en cada columna y si hay valores nulos.

df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rank    1998 non-null   object
 1   Player  1992 non-null   object
 2   Age     1992 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [857]:
# Vamos a cambiar el orden de las columnas para que sea más intuitivo.

df_ranking = df_ranking[["Player", "Age", "Rank"]]
df_ranking.head()


,Player,Age,Rank
0,\n Carlos Alcaraz\n...,\n19,\n 1\n
1,\n Novak Djokovic\n...,\n35,\n 2\n
2,\n Stefanos Tsitsip...,\n24,\n 3\n
3,\n Casper Ruud\n ...,\n24,\n 4\n
4,\n Daniil Medvedev\...,\n27,\n 5\n


In [858]:
# convertimos todas las columnas a float

df_ranking = convert_to_float(df_ranking)
df_ranking.head()

,Player,Age,Rank
0,\n Carlos Alcaraz\n...,19.0,1.0
1,\n Novak Djokovic\n...,35.0,2.0
2,\n Stefanos Tsitsip...,24.0,3.0
3,\n Casper Ruud\n ...,24.0,4.0
4,\n Daniil Medvedev\...,27.0,5.0


In [859]:
# Por ultimo vamos a quitar todas las \n que hay en la columna Player.

df_ranking['Player'] = df_ranking['Player'].apply(lambda x: x.replace('\n', '') if isinstance(x, str) else x)
df_ranking.head()


,Player,Age,Rank
0,Carlos Alcaraz ...,19.0,1.0
1,Novak Djokovic ...,35.0,2.0
2,Stefanos Tsitsipas...,24.0,3.0
3,Casper Ruud ...,24.0,4.0
4,Daniil Medvedev ...,27.0,5.0


In [860]:
# Comprobamos que los datos se han convertido correctamente.

df_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  1992 non-null   object 
 1   Age     1980 non-null   float32
 2   Rank    1188 non-null   float32
dtypes: float32(2), object(1)
memory usage: 31.3+ KB


# MERGE DATAFRAMES

## Comenzamos uniendo los 4 dataframes que contienen las estadísticas de los jugadores para crear un único dataframe.

In [861]:
# Vamos a hacer un merge de los 4 dataframes que contienen las estadísticas de los jugadores.
# Usamos la columna Player como clave para hacer el merge.

# Primero la columna Player la pasamos a formato Title para que coincida con el formato de los otros dataframes.
# Eliminamos si tiene espacios al principio o al final.

df_ranking['Player'] = df_ranking['Player'].apply(lambda x: x.title().strip() if isinstance(x, str) else x)
df_ranking.head()

,Player,Age,Rank
0,Carlos Alcaraz,19.0,1.0
1,Novak Djokovic,35.0,2.0
2,Stefanos Tsitsipas,24.0,3.0
3,Casper Ruud,24.0,4.0
4,Daniil Medvedev,27.0,5.0


In [862]:
# Hacemos el merge de los 4 dataframes.

df_stats = pd.merge(df_serve, df_return, on='Player')
df_stats = pd.merge(df_stats, df_underpressure, on='Player')
df_stats = pd.merge(df_stats, df_win_loss, on='Player')
df_stats.head()

,Player,1stServe,1stServePointsWon,2ndServePointsWon,ServiceGamesWon,AvgAcesMatch,AvgDoubleFaultsMatch,1stServeReturnPointsWon,2ndServeReturnPointsWon,ReturnGamesWon,BreakPointsConverted_return,BreakPointsConverted_pressure,BreakPointsSavedPressure,TieBreaksWon,DecidingSetsWon,AvgWinLoss
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,93.800003,20.900000,40.000000,7.100000,219.100006,25.799999,69.300003,71.099998,0.444
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,125.300003,27.200001,46.599998,16.500000,226.399994,35.000000,67.699997,54.700001,0.684
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,129.800003,29.500000,45.700001,18.799999,210.000000,35.799999,67.099998,50.000000,0.500
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,160.500000,33.500000,55.900002,29.299999,249.699997,41.799999,64.300003,80.000000,0.938
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,118.500000,23.799999,42.099998,13.300000,209.000000,39.299999,70.500000,56.299999,0.545


In [863]:
# hacemos lo mismo con el dataframe de ranking.

df_stats['Player'] = df_stats['Player'].apply(lambda x: x.title().strip() if isinstance(x, str) else x)
df_stats.head()

,Player,1stServe,1stServePointsWon,2ndServePointsWon,ServiceGamesWon,AvgAcesMatch,AvgDoubleFaultsMatch,1stServeReturnPointsWon,2ndServeReturnPointsWon,ReturnGamesWon,BreakPointsConverted_return,BreakPointsConverted_pressure,BreakPointsSavedPressure,TieBreaksWon,DecidingSetsWon,AvgWinLoss
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,93.800003,20.900000,40.000000,7.100000,219.100006,25.799999,69.300003,71.099998,0.444
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,125.300003,27.200001,46.599998,16.500000,226.399994,35.000000,67.699997,54.700001,0.684
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,129.800003,29.500000,45.700001,18.799999,210.000000,35.799999,67.099998,50.000000,0.500
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,160.500000,33.500000,55.900002,29.299999,249.699997,41.799999,64.300003,80.000000,0.938
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,118.500000,23.799999,42.099998,13.300000,209.000000,39.299999,70.500000,56.299999,0.545


In [864]:
# Hacemos el merge de los 2 dataframes.

df = pd.merge(df_stats, df_ranking, on='Player')
df.head()

,Player,1stServe,1stServePointsWon,2ndServePointsWon,ServiceGamesWon,AvgAcesMatch,AvgDoubleFaultsMatch,1stServeReturnPointsWon,2ndServeReturnPointsWon,ReturnGamesWon,BreakPointsConverted_return,BreakPointsConverted_pressure,BreakPointsSavedPressure,TieBreaksWon,DecidingSetsWon,AvgWinLoss,Age,Rank
0,John Isner,308.500000,67.699997,79.800003,52.700001,90.199997,21.1,93.800003,20.900000,40.000000,7.100000,219.100006,25.799999,69.300003,71.099998,0.444,37.0,46.0
1,Hubert Hurkacz,299.700012,63.400002,79.199997,54.900002,89.900002,14.0,125.300003,27.200001,46.599998,16.500000,226.399994,35.000000,67.699997,54.700001,0.684,26.0,9.0
2,Matteo Berrettini,292.200012,62.500000,79.599998,52.000000,88.500000,11.5,129.800003,29.500000,45.700001,18.799999,210.000000,35.799999,67.099998,50.000000,0.500,26.0,23.0
3,Novak Djokovic,292.100006,65.300003,77.199997,57.000000,88.699997,6.0,160.500000,33.500000,55.900002,29.299999,249.699997,41.799999,64.300003,80.000000,0.938,35.0,2.0
4,Ben Shelton,292.000000,60.700001,79.300003,55.900002,89.400002,11.3,118.500000,23.799999,42.099998,13.300000,209.000000,39.299999,70.500000,56.299999,0.545,20.0,39.0
